In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
data = pd.read_csv('C:\\Users\\Administrator\\Documents\\kaggle\\train.csv\\train.csv')

In [4]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)